In [4]:
import geopandas as gpd
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
import pulp

shapefile_path = "land_parcels.shp"
gdf = gpd.read_file(shapefile_path)
# reproject to EPSG:3347
gdf = gdf.to_crs(epsg=3347)

# compute area in square kilometers
gdf["area_km2"] = gdf.geometry.area / 1e6

# define a reasonable area threshold (e.g., 0.1 km²)
area_threshold = 50
init_count = gdf.shape[0]
gdf = gdf[gdf["area_km2"] >= area_threshold]
filtered_count = init_count - gdf.shape[0]

# summary statistics
num_polygons = gdf.shape[0]
min_area = gdf["area_km2"].min()
avg_area = gdf["area_km2"].mean()
max_area = gdf["area_km2"].max()
carbon_store_range = (gdf["carbon_sto"].min(), gdf["carbon_sto"].max())
cost_range = (gdf["cost"].min(), gdf["cost"].max())

print(f"Number of polygons after filtering: {num_polygons}")
print(f"Polygons removed: {filtered_count}")
print(f"Min area (km²): {min_area}")
print(f"Avg area (km²): {avg_area}")
print(f"Max area (km²): {max_area}")
print(f"Carbon store range: {carbon_store_range}")
print(f"Cost range: {cost_range}")

Number of polygons after filtering: 95
Polygons removed: 5
Min area (km²): 53.05923376683125
Avg area (km²): 82.44460022848595
Max area (km²): 132.0955428630866
Carbon store range: (10.036534572755878, 99.8608996768998)
Cost range: (524.9516816211429, 4982.434015347925)


In [5]:
# compute adjacency matrix
num_parcels = len(gdf)
adjacency_matrix = np.zeros((num_parcels, num_parcels), dtype=int)
for i in range(num_parcels):
    for j in range(i + 1, num_parcels):
        if gdf.geometry.iloc[i].touches(gdf.geometry.iloc[j]):
            adjacency_matrix[i, j] = 1
            adjacency_matrix[j, i] = 1
print(adjacency_matrix.shape)

(95, 95)


In [7]:
print(pulp.listSolvers(onlyAvailable=True))
# define the problem
problem = LpProblem(name="land_parcel_optimization", sense=LpMaximize)

# define decision Variables
parcels = list(gdf.index)
x = {p: LpVariable(name=f"parcel_{p}", cat='Binary') for p in parcels}

# Constraints
budget_limit = 0.5 * gdf["cost"].sum()
problem += lpSum(gdf.loc[p, "cost"] * x[p] for p in parcels) <= budget_limit

for i in range(num_parcels):
    for j in range(num_parcels):
        if adjacency_matrix[i, j] == 1:
            problem += x[parcels[i]] + x[parcels[j]] <= 1

# Objective Function
problem += lpSum(gdf.loc[p, "carbon_sto"] * x[p] for p in parcels)

# Step 4: Solve the Problem
problem.solve()

# Step 5: Output Results
selected_parcels = [p for p in parcels if x[p].varValue == 1]
total_carbon_store = sum(gdf.loc[p, "carbon_sto"] for p in selected_parcels)
total_cost = sum(gdf.loc[p, "cost"] for p in selected_parcels)
total_carbon_store, total_cost

['GLPK_CMD']
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --cpxlp /var/folders/d4/6r524yqx4lsbn99gww8xth600000gn/T/e5685009724d4caeb5b8b834562f4d40-pulp.lp
 -o /var/folders/d4/6r524yqx4lsbn99gww8xth600000gn/T/e5685009724d4caeb5b8b834562f4d40-pulp.sol
Reading problem data from '/var/folders/d4/6r524yqx4lsbn99gww8xth600000gn/T/e5685009724d4caeb5b8b834562f4d40-pulp.lp'...
625 rows, 95 columns, 1343 non-zeros
95 integer variables, all of which are binary
788 lines were read
GLPK Integer Optimizer 5.0
625 rows, 95 columns, 1343 non-zeros
95 integer variables, all of which are binary
Preprocessing...
625 rows, 95 columns, 1343 non-zeros
95 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  4.982e+03  ratio =  4.982e+03
GM: min|aij| =  5.728e-01  max|aij| =  1.746e+00  ratio =  3.047e+00
EQ: min|aij| =  3.291e-01  max|aij| =  1.000e+00  ratio =  3.039e+00
2N: min|aij| =  1.772e-01  max|aij| =  1.000e+00  ratio =  5.642e+

(1864.9355125204672, 63218.08899804288)